# Published data - Cupriavidus necator
Data collection
- Batch culture samples from two time points during exponential phase

In [1]:
from BFAIR.mfa.INCA import INCA_script
import pandas as pd
import numpy as np
import time
import ast
import matlab.engine
import sys
#import escher
import dotenv
from BFAIR.mfa.INCA import INCA_reimport
from BFAIR.parsers import modelReactions_file_parser, atomMapping_reactions2_file_parser, atom_mapping_metabolites_file_parser

DEBUG:optlang.util:Gurobi python bindings not available.
DEBUG:optlang.util:GLPK python bindings found at /Users/s143838/.virtualenvs/bfair-testing/lib/python3.10/site-packages/swiglpk
DEBUG:optlang.util:Mosek python bindings not available.
DEBUG:optlang.util:CPLEX python bindings not available.
DEBUG:optlang.util:OSQP python bindings not available.
DEBUG:optlang.util:COINOR_CBC python bindings not available.
DEBUG:optlang.util:Scipy linprog function found at /Users/s143838/.virtualenvs/bfair-testing/lib/python3.10/site-packages/scipy/optimize/__init__.py


In [2]:
# import environment variables
INCA_base_directory = dotenv.get_key(dotenv.find_dotenv(), "INCA_base_directory")

#### Import using parsers

In [3]:
imported_reactions = modelReactions_file_parser(
    'Literature data/Cupriavidus necator  Alagesan 2017/reactions_2nd.xlsx',
    'Cupriavidus_necator2017',
    reaction_id_col_name="Reaction ID",
    equation_col_name="Equations (Carbon atom transition)",
)
imported_atom_mapping = atomMapping_reactions2_file_parser(
    'Literature data/Cupriavidus necator  Alagesan 2017/reactions_2nd.xlsx',
    'Cupriavidus_necator2017',
    reaction_id_col_name="Reaction ID",
    equation_col_name="Equations (Carbon atom transition)",
)
imported_atoms_metabolites = atom_mapping_metabolites_file_parser(
    'Literature data/Cupriavidus necator  Alagesan 2017/reactions_2nd.xlsx',
    'Cupriavidus_necator2017',
    reaction_id_col_name="Reaction ID",
    equation_col_name="Equations (Carbon atom transition)",
)

DEBUG:BFAIR.parsers.data_import_parsers:Duplicates in reactants: [('PYR', 1.0, 'abc'), ('PYR', 1.0, 'def')]
DEBUG:BFAIR.parsers.data_import_parsers:Duplicates: ['PYR']
DEBUG:BFAIR.parsers.data_import_parsers:Duplicate mappings: ['abc', 'def']
DEBUG:BFAIR.parsers.data_import_parsers:Duplicates in reactants: [('PYR', 1.0, 'abc'), ('PYR', 1.0, 'def')]
DEBUG:BFAIR.parsers.data_import_parsers:Duplicates: ['PYR']
DEBUG:BFAIR.parsers.data_import_parsers:Duplicate mappings: ['abc', 'def']


#### Manual set up of additional information

In [4]:
tracer_info = pd.DataFrame.from_dict({
    'experiment_id': [
        'D-[1-13C]fructose', '[1,2-13C]glycerol', '[1,2-13C]glycerolandCO2',
    ],
    'met_id': ['F6P.ext', 'GLY.ext', 'GLY.ext'],
    'met_name': [
        'D-[1-13C]fructose', '[1,2-13C]glycerol', '[1,2-13C]glycerolandCO2',
    ],
    'met_atompositions': [
        '{1}', '{1,2}', '{1,2}',
    ],
    'met_elements': [
        '{C}', '{C,C}', '{C,C}',
    ],
    'ratio': [
        '1', '1', '1',
    ]    
}, orient='columns')

In [5]:
measured_fluxes_info = pd.DataFrame.from_dict({
    'experiment_id': [
        'D-[1-13C]fructose', '[1,2-13C]glycerol', '[1,2-13C]glycerolandCO2',
    ],
    'model_id': ['Cupriavidus_necator2017' for i in range(3)],
    'rxn_id': [
        'ex_1', 'ex_2', 'ex_2',
        ],
    'flux_average': ['1' for i in range(3)],
    'flux_stdev': ['0.01' for i in range(3)],
    'flux_lb': ['0.99' for i in range(3)],
    'flux_ub': ['1.01' for i in range(3)],  
}, orient='columns')

#### This one should be its own parser

In [6]:
experimental_mdvs = pd.read_csv('Literature data/Cupriavidus necator  Alagesan 2017/MDVs.csv', sep=';', decimal=',')

In [7]:
experiment_ids = []
met_ids = []
fragment_ids = []
time_points = []
intensity_normalized_averages = []
intensity_normalized_stdevs = []
met_atompositions = []
met_elements = []
sample_name_abbreviation = []

experiments = [experimental_mdvs.columns[2], experimental_mdvs.columns[4], experimental_mdvs.columns[6]]
for experiment in experiments:
    fragments = list(set([row[1] for i, row in experimental_mdvs[experimental_mdvs.columns[:2]].iterrows()]))
    for fragment in fragments:
        experiment_ids.append(experiment)
        fragment_ids.append(fragment)
        mdv_list = experimental_mdvs[experimental_mdvs[experimental_mdvs.columns[1]] == fragment][experiment].to_list()
        stdev_list = experimental_mdvs[experimental_mdvs[experimental_mdvs.columns[1]] == fragment]['stdev_' + experiment].to_list()
        mdv = '{'
        stdev = '{'
        atompos = '{'
        frag_element = '{'
        for i, mdv_value in enumerate(mdv_list):
            if i == 0:
                mdv += str(mdv_value)
                stdev += str(stdev_list[i])
            else:
                mdv += ',' + str(mdv_value)
                stdev += ',' + str(stdev_list[i])
                if i == 1:
                    atompos += str(i-1)
                    frag_element += 'C'
                else:
                    atompos += ',' + str(i-1)
                    frag_element += ',' + 'C'
        mdv += '}'
        stdev += '}'
        atompos += '}'
        frag_element += '}'

        met_ids.append(list(set(experimental_mdvs[experimental_mdvs[experimental_mdvs.columns[1]] == fragment]['Met'].to_list()))[0])
        time_points.append('0')
        intensity_normalized_averages.append(mdv)
        intensity_normalized_stdevs.append(stdev)
        sample_name_abbreviation.append('')
        met_atompositions.append(atompos)
        met_elements.append(frag_element)

mdvs = pd.DataFrame.from_dict({
    'experiment_id': experiment_ids,
    'met_id': met_ids,
    'fragment_id': fragment_ids,
    'time_point': time_points,
    'met_atompositions': met_atompositions,
    'met_elements': met_elements,
    'sample_name_abbreviation': sample_name_abbreviation,
    'intensity_normalized_average': intensity_normalized_averages,
    'intensity_normalized_stdev': intensity_normalized_stdevs,  
}, orient='columns')

In [8]:
mdvs

,experiment_id,met_id,fragment_id,time_point,met_atompositions,met_elements,sample_name_abbreviation,intensity_normalized_average,intensity_normalized_stdev
0,D-[1-13C]fructose,HIS,Histidine338,0,"{0,1,2,3,4}","{C,C,C,C,C}",,"{0.4626,0.4672,0.0427,0.0255,0.0008,0.0009}","{0.0091,0.0173,0.0119,0.0030,0.001,nan}"
1,D-[1-13C]fructose,ALA,Alanine232,0,"{0,1}","{C,C}",,"{0.9759,0.0230,0.0011}","{0.0027,0.0035,0.0009}"
2,D-[1-13C]fructose,ILE,Isoleucine274,0,"{0,1,2,3,4}","{C,C,C,C,C}",,"{0.8210975,0.1654325,0.01305,0.0003375,0,0}","{0.0120,0.0083,0.0040,0.001,0.001,0.001}"
3,D-[1-13C]fructose,THR,Threonine404,0,"{0,1,2,3}","{C,C,C,C}",,"{0.7867,0.1954,0.0161,0.0017,0}","{0.0107,0.0125,0.0056,0.001,0.001}"
4,D-[1-13C]fructose,GLU,Glutamicacid432,0,"{0,1,2,3,4}","{C,C,C,C,C}",,"{0.8192,0.1679,0.0125,0.0002,0.0001,0}","{0.0129,0.0102,0.0028,0.001,0.001,0.001}"
...,...,...,...,...,...,...,...,...,...
64,"[1,2-13C]glycerolandCO2",ALA,Alanine260,0,"{0,1,2}","{C,C,C}",,"{0.1996,0.3592,0.4174,0.0238}","{0.0337,0.0214,0.0544,0.0124}"
65,"[1,2-13C]glycerolandCO2",PHE,Phenylalanine302,0,"{0,1}","{C,C}",,"{0.2943,0.6091,0.0965}","{0.0338,0.0236,0.0242}"
66,"[1,2-13C]glycerolandCO2",MET,Methionine320,0,"{0,1,2,3,4}","{C,C,C,C,C}",,"{0.1144,0.2522,0.3458,0.2137,0.0634,0.0105}","{0.0203,0.0259,0.0147,0.0306,0.0147,0.0009}"
67,"[1,2-13C]glycerolandCO2",GL,Glycine246,0,"{0,1}","{C,C}",,"{0.3019,0.6202,0.0778}","{0.0322,0.0331,0.0263}"


In [9]:
# measured fragments/MS data, tracers and measured fluxes should be limited to one experiment

atomMappingReactions_data_I = imported_atom_mapping
modelReaction_data_I = imported_reactions
atomMappingMetabolite_data_I = imported_atoms_metabolites
measuredFluxes_data_I = measured_fluxes_info
experimentalMS_data_I = mdvs
tracer_I = tracer_info

In [10]:
INCA_script_test2 = INCA_script()
# The files need to be limited by model id and mapping id, I picked "ecoli_RL2013_02" here
atomMappingReactions_data_I = INCA_script_test2.limit_to_one_model(atomMappingReactions_data_I, 'mapping_id', 'Cupriavidus_necator2017')
modelReaction_data_I = INCA_script_test2.limit_to_one_model(modelReaction_data_I, 'model_id', 'Cupriavidus_necator2017')
atomMappingMetabolite_data_I = INCA_script_test2.limit_to_one_model(atomMappingMetabolite_data_I, 'mapping_id', 'Cupriavidus_necator2017')
measuredFluxes_data_I = INCA_script_test2.limit_to_one_model(measuredFluxes_data_I, 'model_id', 'Cupriavidus_necator2017')

# Limiting fluxes, fragments and tracers to one experiment
measuredFluxes_data_I = INCA_script_test2.limit_to_one_experiment(measuredFluxes_data_I, 'experiment_id', '[1,2-13C]glycerol')
experimentalMS_data_I = INCA_script_test2.limit_to_one_experiment(experimentalMS_data_I, 'experiment_id', '[1,2-13C]glycerol')
tracer_I = INCA_script_test2.limit_to_one_experiment(tracer_I, 'experiment_id', '[1,2-13C]glycerol')

In [11]:
script = INCA_script_test2.script_generator(
    modelReaction_data_I,
    atomMappingReactions_data_I,
    atomMappingMetabolite_data_I,
    measuredFluxes_data_I,
    experimentalMS_data_I,
    tracer_I
)

script_folder = %pwd
matlab_script = "c_necator_lit"
runner_script = matlab_script + "_runner"
INCA_script_test2.save_INCA_script(script, matlab_script)
runner = INCA_script_test2.runner_script_generator('C_necator', 10)
INCA_script_test2.save_runner_script(runner=runner, scriptname=matlab_script)

There is no stoichiometry given for: R72


In [12]:
# Comment out to avoid rerunning INCA
#INCA_script_test2.run_INCA_in_MATLAB(INCA_base_directory, script_folder, matlab_script, runner_script)

In [13]:
filename = 'C_necator.mat'
simulation_info = experimentalMS_data_I
simulation_id = '[1,2-13C]glycerol'

In [14]:
experimentalMS_data_I.experiment_id.unique()

array(['[1,2-13C]glycerol'], dtype=object)

In [15]:
from BFAIR.mfa.INCA.INCA_results import INCA_results

In [16]:
res = INCA_results(filename)

In [17]:
res.measurements_and_fit_overview

,cont,expt,id,sres,type,base
0,"[0.06449457955047126, 0.9999984177011418, 0.99...","[1,2-13C]glycerol",ex_2,0.004346,Flux,{'id': []}
1,"[0.21889214540609847, 4.3020516702422165e-08, ...","[1,2-13C]glycerol",Alanine232,99.805644,MS,{'id': []}
2,"[0.005128644242374778, 2.560123758006283e-08, ...","[1,2-13C]glycerol",Alanine260,780.054728,MS,{'id': []}
3,"[0.06640039130206483, 1.0630176151281687e-07, ...","[1,2-13C]glycerol",Asparticacid302,69.624349,MS,{'id': []}
4,"[0.03279228167786573, 7.440891423737781e-09, 7...","[1,2-13C]glycerol",Asparticacid390,55.760320,MS,{'id': []}
5,"[0.013820855543442749, 1.834418519048734e-08, ...","[1,2-13C]glycerol",Asparticacid418,547.216785,MS,{'id': []}
6,"[0.1335125529336202, 1.559382705105003e-07, 1....","[1,2-13C]glycerol",Glutamicacid330,1323.495663,MS,{'id': []}
7,"[0.07897074128089261, 5.497147862848788e-09, 5...","[1,2-13C]glycerol",Glutamicacid432,347.631472,MS,{'id': []}
8,"[0.002462002178489449, 1.5831968603338503e-08,...","[1,2-13C]glycerol",Glycine218,3150.105509,MS,{'id': []}
9,"[0.03997380425518976, 6.831394983032977e-08, 6...","[1,2-13C]glycerol",Glycine246,582.397578,MS,{'id': []}


Now that we have loaded the inca results, we can investigate how results. The first step is to investigate the diagnostics. Here we want to investigate a few factors:
1. Did the fit pass the Goodness-of-fit test
2. Are the residuals normally distributed
3. Are the any measurements that appears to be outliers

 

In [18]:
res.get_goodness_of_fit()

Fit accepted: False
Confidence level: 0.05
Chi-square value (SSR): 13780.709741407143
Expected chi-square range: [33.16178637 72.61599227]


In [20]:
res.test_normality()

Normality test: False


In [19]:
import BFAIR.mfa.visualization.diagnositics as diagnostics

diagnostics.plot_norm_probplot(res, interactive=True)

DEBUG:matplotlib:matplotlib data path: /Users/s143838/.virtualenvs/bfair-testing/lib/python3.10/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/Users/s143838/.matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is darwin
DEBUG:matplotlib:CACHEDIR=/Users/s143838/.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /Users/s143838/.matplotlib/fontlist-v330.json
/Users/s143838/.virtualenvs/bfair-testing/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

The issue is that the parser for the parameter info relies on very specific naming of the fragments, i.e. it relies on the id being split and hardcoded indexing to select different features. 

I happens because the `len(fragment_list)` parameter is < 5. But in general this data does not contain the compound equation for the fragments. The best solution would probably to remake the data model for how the information, which is currently stored in the rxn_id, could be more properly handled.

The `rxn_id` is parsed directly from the matlab object in `get_fitted_parameters()`. Thus, its is matlab/INCA which creates these strange ids.

The fragment data does not contain fragment formular, I think this is the reason why it fails.

The msdata() has the .more attribute, this appears to contain the atoms of all non-labelled atoms, this could for example be all non carbon atoms in the molecule or the fomular for the derivatized compound minus the carbon atoms that originate from the amino acid.


## Refactoring sort_... functions
These functions are hard to understand because they do many thing at the same time. Their main purpose seems to be converting a dictionary into a pd.Dataframe(). This can be done much simpler using pd.DataFrame.from_dict(). Then it only remain to add a bit of extra information, such as simulation_id and simulation_dateAndTime.

We see that it is only the 'simulation_id', 'simulation_dateAndTime' that are missing.

Expected output columns fittedMeasuredFragmentsResiduals (Found in the MFA_INCA_data_reimport.ipynb):

['simulation_id', MISSING
'simulation_dateAndTime', MISSING
'experiment_id',
'sample_name_abbreviation', MISSING
'time_point', 
'fragment_id',MISSING
'fragment_mass', MISSING
'res_data', 
'res_fit', 
'res_peak', 
'res_stdev',
'res_val',
'res_msens', MISSING - but hardcoded to None
'res_esens', MISSING - but hardcoded to None
'used_', MISSING - but hardcoded to True
'comment_']MISSING - but hardcoded to None

The reaction ID contain the fragment id, this is due to the specific id schema for the fragments. Thus it should be safe to take the fragment ID from the reation ID.